In [ ]:
from sklearn import preprocessing
import http.client, urllib.parse
import pandas as pd
import numpy as np
import math
import json

In [ ]:
def getDist(houses, locations):
    
    min_dist = 999999999
    
    for house in houses:
        x1 = house['latitude']
        x2 = house['longitude']
        for location in locations:
            x2 = location['latitude']
            y2 = location['longitude']
            dist = (math.sqrt((x2 - x1)**2 + (y2 - y1)**2))*100
            if dist < min_dist:
                min_dist = dist
                
        distances.append(min_dist)  
    
    return distances

def getCoords(df):
    
    #api info
    conn = http.client.HTTPConnection('api.positionstack.com')
    key = 'df3b33e27a0f6451fd9aae993c6a26fa'
    
    
    def query(coord, address):

        params = urllib.parse.urlencode({
            'access_key': key,
            'query': address,
            'limit': 1
            })
        
        conn.request('GET', '/v1/forward?{}'.format(params))
        results = conn.getresponse()
        data = json.loads(results.read())['data'][0] #converts json to dict
        
        if coord == 'x':
            result = data['latitude']
        else:
            result = data['longitude']
        
        return result
    

    lat = [query('x', address) for address in df.iloc[:,0]]
    lon = [query('y', address) for address in df.iloc[:,0]]
    
    df['latitude'] = lat
    df['longitude'] = lon
    
    return df

In [ ]:
#IMPORT AND ORGANIZE DATA
#get coordinates when necessary

houses = pd.read_csv('/data/houses_clean.csv')

houses_la = houses.subset(houses['City'] == 'Los Angeles'.tolower())
houses_anaheim = houses.subset(houses['City'] == 'Anaheim')
houses_longbeach = houses.subset(houses['City'] == 'Long Beach')

houses_dallas = houses.subset(houses['City'] == 'Dallas')
houses_fortworth = houses.subset(houses['City'] == 'Fort Worth')
houses_arlington_tx = houses.subset(houses['City'] == 'Arlington' and houses['State'] == 'TX')

houses_chicago = houses.subset(houses['City'] == 'Chicago')
houses_naperville = houses.subset(houses['City'] == 'Naperville')
houses_elgin = houses.subset(houses['City'] == 'Elgin')

houses_washington = houses.subset(houses['City'] == 'Washington')
houses_alexandria = houses.subset(houses['City'] == 'Alexandria')
houses_arlington_va = houses.subset(houses['City'] == 'Arlington' and houses['State'] == 'VA')

schools = pd.read_csv('/data/schools_clean.cvs')
schools = getCoords(schools)
schools_la = schools.subset(schools['City'] == 'Los Angeles'.tolower())
schools_anaheim = schools.subset(schools['City'] == 'Anaheim')
schools_longbeach = schools.subset(schools['City'] == 'Long Beach')

schools_dallas = schools.subset(schools['City'] == 'Dallas')
schools_fortworth = schools.subset(schools['City'] == 'Fort Worth')
schools_arlington_tx = schools.subset(schools['City'] == 'Arlington' and schools['State'] == 'TX')

schools_chicago = schools.subset(schools['City'] == 'Chicago')
schools_naperville = schools.subset(schools['City'] == 'Naperville')
schools_elgin = schools.subset(schools['City'] == 'Elgin')

schools_washington = schools.subset(schools['City'] == 'Washington')
schools_alexandria = schools.subset(schools['City'] == 'Alexandria')
schools_arlington_va = schools.subset(schools['City'] == 'Arlington' and schools['State'] == 'VA')

#...

other = pd.read_csv('data/other_locations.csv')
other = getCoords(other)
hospitals_la = other.subset(other['City'].find() >= 0)['Hospitals']
#...

In [ ]:
#NORMALIZE HOUSE PRICES ACCORDING TO REGION

#combine cities into metro areas
houses_ca = pd.concat([houses_la, houses_anaheim, houses_longbeach])
houses_tx = pd.concat([houses_dallas, houses_fortworth, houses_arlington_tx])
houses_dc = pd.concat([houses_washington, houses_alexandria, houses_arlington_va])
houses_il = pd.concat([houses_chicago, houses_naperville, houses_elgin])

#convert prices into numpy arrays
price_ca = np.array(houses_ca['Price'])
price_tx = np.array(houses_tx['Price'])
price_dc = np.array(houses_dc['Price'])
price_il = np.array(houses_il['Price'])

#normalize
price_ca = preprocessing.normalize([price_ca])
price_tx = preprocessing.normalize([price_tx])
price_dc = preprocessing.normalize([price_dc])
price_il = preprocessing.normalize([price_il])

#replace prices back into their respective dataframes
houses_ca['Price'] = price_ca
houses_tx['Price'] = price_tx
houses_dc['Price'] = price_dc
houses_il['Price'] = price_il

#subset so that all city dataframes have the normalized price values
houses_la = houses_ca.subset(houses['City'] == 'Los Angeles'.tolower())
houses_anaheim = houses_ca.subset(houses['City'] == 'Anaheim')
houses_longbeach = houses_ca.subset(houses['City'] == 'Long Beach')

houses_dallas = houses_tx.subset(houses['City'] == 'Dallas')
houses_fortworth = houses_tx.subset(houses['City'] == 'Fort Worth')
houses_arlington_tx = houses_tx.subset(houses['City'] == 'Arlington' and houses['State'] == 'TX')

houses_chicago = houses_il.subset(houses['City'] == 'Chicago')
houses_naperville = houses_il.subset(houses['City'] == 'Naperville')
houses_elgin = houses_il.subset(houses['City'] == 'Elgin')

houses_washington = houses_va.subset(houses['City'] == 'Washington')
houses_alexandria = houses_va.subset(houses['City'] == 'Alexandria')
houses_arlington_va = houses_va.subset(houses['City'] == 'Arlington' and houses['State'] == 'VA')

In [ ]:
#ADJUST HOUSE PRICES BY SQUARE FOOTAGE

In [ ]:
#GET DISTANCES

houses_la['dist_hospitals'] = getDist(houses_la, hositals_la)
#...

houses_dallas['dist_gyms'] = getDist(houses_dallas, gyms_dallas)
#...

In [ ]:
#PERFORM ANALYSIS USING DISTANCES AS VARIABLES TO PREDICT PRICE